# Домашняя работа #2
Построим двумя способами доверительные интервалы для дисперсии нормального распределения, когда математиеское ожидание известно (будем считать, что оно равно нулю)

С помощью статистики $\sum{\frac{X_k^2}{n}}$ получаем интервал:
$$
(\frac{\sum X_k^2}{\lambda_{(1+\gamma)/2}}, \frac{\sum X_k^2}{\lambda_{(1-\gamma)/2}})
$$

где $\lambda_p$ - квантиль порядка $p$ распределения хи-квадрат с $n$ степенями свободы.

С помощью статистики $(\sum{\frac{X_k}{n}})^2$ получаем интервал:
$$
(\frac{n\overline X_k^2}{z_{(3+\gamma)/4}^2}, \frac{n\overline X_k^2}{z_{(3+\gamma)/4}^2})
$$

где $z_p$ - квантиль порядка $p$ стандартного нормального распределения.

In [ ]:
import numpy as np
from scipy.stats import chi2, norm
import plotly.graph_objs as go
import plotly


plotly.offline.init_notebook_mode()


def st1(samp: np.ndarray):
    gamma = 0.95
    sq_sum = np.sum(samp ** 2)
    left_quantile = chi2.ppf((1 + gamma) / 2, len(samp))
    right_quantile = chi2.ppf((1 - gamma) / 2, len(samp))

    l = sq_sum / left_quantile
    r = sq_sum / right_quantile

    return l, r


def st2(samp: np.ndarray):
    gamma = 0.99
    m = np.mean(samp)
    n = len(samp)

    left_quantile = norm.ppf((3 + gamma) / 4)
    right_quantile = norm.ppf((3 - gamma) / 4)

    l = n * m ** 2 / left_quantile ** 2
    r = n * m ** 2 / right_quantile ** 2

    return l, r

Посмотрим как меняется интервал в зависимости от размера выборки для первой статтистики.

In [ ]:
def gen_sample(n, sigma):
    return np.random.normal(0, sigma, n)


def test_stat(sigma, st, l, r, step):
    l_arr, r_arr, sig, x = [], [], [], []
    for i in range(l, r + 1, step):
        sample = gen_sample(i, sigma)
        sigma_l, sigma_r = st(sample)

        x.append(i)
        l_arr.append(sigma_l)
        r_arr.append(sigma_r)
        sig.append(sigma ** 2)
    return l_arr, r_arr, sig, x


def plot(*data):
    traces = []
    x = data[-1]
    colors = [
        {'color': 'rgba(0, 255, 0, 0.8)'},
        {'color': 'rgba(255, 0, 0, 0.8)'},
        {'color': 'rgba(0, 0, 0, 0.8)'},
    ]
    names = [
        'left',
        'right',
        f'$\sigma^2$'
    ]
    for i in range(len(data) - 1):
        traces.append(go.Scatter(
            x=x,
            y=data[i],
            mode='lines+markers',
            marker=colors[i],
            name=names[i]
        ))

    layout = go.Layout(
        title={
            'text': "Зависимость границ интервала от размера выборки",
            'y': 0.9,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis={
            'title': 'n'
        },
        yaxis={
            'title': f'$\sigma^2$'
        }
    )
    fig = go.Figure(data=traces, layout=layout)
    plotly.offline.iplot(fig)

sigma = 10
data = test_stat(sigma, st1, 50, 10000, 50)
plot(*data)

На графике видна четкая зависимость размера интервала от размера выборки -- чем больше выборка, тем меньше интервал.

Теперь посмотрим на 2 статистику.

In [ ]:
data = test_stat(sigma, st2, 50, 10000, 50)
plot(*data)

Левая граница полностью совпадает с дисперсией, а правая совершенно не зависит от размера. В целом то, что статистика плоха, можно также объяснить тем, что $\overline X^2$ стремится к 0 по ЗБЧ, поэтому оценивать с помощью этой статистики точно не является лучшим решением.